In [ ]:
import os
import re
import glob
from datetime import datetime
import sys
import itertools
from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
sys.path.append("..") # Adds higher directory to python modules path for importing from src dir

from src.algorithms import SimpleLSTM, MLPClassifier, SVC
from src.datasets import NyseStocksDataset
from src.evaluation import Evaluator
from src.utils import print_metrics_as_md, pandas_df_to_markdown_table
# from src.pipeline import run_pipeline, prepare_data, evaluate_results
# from src.experiments import get_metrics

%matplotlib inline
%load_ext autotime
%load_ext autoreload
%autoreload 2

## 1. 15 runs

- SimpleLSTM with seeds from 42 to 56
- On OC-3C and OC-3C+
- 90k train, 9k test
- 512 batch size

| -  |   datasets    predictors     |     OC-3C    SLSTM_42   |     OC-3C    SLSTM_43   |     OC-3C    SLSTM_44   |     OC-3C    SLSTM_45   |     OC-3C    SLSTM_46   |     OC-3C    SLSTM_47   |     OC-3C    SLSTM_48   |     OC-3C    SLSTM_49   |     OC-3C    SLSTM_50   |     OC-3C    SLSTM_51   |     OC-3C    SLSTM_52   |     OC-3C    SLSTM_53   |     OC-3C    SLSTM_54   |     OC-3C    SLSTM_55   |     OC-3C    SLSTM_56   |     OC-3C+    SLSTM_42   |     OC-3C+    SLSTM_43   |     OC-3C+    SLSTM_44   |     OC-3C+    SLSTM_45   |     OC-3C+    SLSTM_46   |     OC-3C+    SLSTM_47   |     OC-3C+    SLSTM_48   |     OC-3C+    SLSTM_49   |     OC-3C+    SLSTM_50   |     OC-3C+    SLSTM_51   |     OC-3C+    SLSTM_52   |     OC-3C+    SLSTM_53   |     OC-3C+    SLSTM_54   |     OC-3C+    SLSTM_55   |     OC-3C+    SLSTM_56   |
|---:|:-----------------------------|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|-------------------------:|
|  0 | prec                         |                 0.36441 |                 0.35397 |                 0.38462 |                 0.36813 |                 0.37811 |                 0.37419 |                 0.38523 |                 0.34052 |                 0.3722  |                 0.39108 |                 0.35703 |                 0.38181 |                 0.36864 |                 0.34628 |                 0.37142 |                  0.22822 |                  0.32555 |                  0.36945 |                  0.36879 |                  0.31527 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |
|  1 | rec                          |                 0.37278 |                 0.35878 |                 0.381   |                 0.37911 |                 0.38744 |                 0.38322 |                 0.37844 |                 0.36167 |                 0.37011 |                 0.38311 |                 0.37022 |                 0.37833 |                 0.37867 |                 0.36544 |                 0.37789 |                  0.34022 |                  0.33311 |                  0.37656 |                  0.37322 |                  0.34011 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |
|  2 | f1                           |                 0.29667 |                 0.24276 |                 0.29404 |                 0.31864 |                 0.3297  |                 0.31781 |                 0.2985  |                 0.26826 |                 0.2745  |                 0.29808 |                 0.28126 |                 0.29798 |                 0.31248 |                 0.28542 |                 0.30624 |                  0.19255 |                  0.2956  |                  0.29925 |                  0.28082 |                  0.19564 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |
|  3 | acc                          |                 0.37278 |                 0.35878 |                 0.381   |                 0.37911 |                 0.38744 |                 0.38322 |                 0.37844 |                 0.36167 |                 0.37011 |                 0.38311 |                 0.37022 |                 0.37833 |                 0.37867 |                 0.36544 |                 0.37789 |                  0.34022 |                  0.33311 |                  0.37656 |                  0.37322 |                  0.34011 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |
|  4 | mcc                          |                 0.07882 |                 0.06828 |                 0.10221 |                 0.08302 |                 0.09732 |                 0.09366 |                 0.094   |                 0.06191 |                 0.08404 |                 0.10601 |                 0.07855 |                 0.09329 |                 0.08554 |                 0.06459 |                 0.08684 |                  0.03069 |                 -0.00037 |                  0.08678 |                  0.08774 |                  0.02772 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |                        0 |


**Outcome**: Execution became very slow after a few iterations. Had to stop after 20 out of 30. Nevertheless OC-3C+ appears to be worse. The histories show that a the accuracy is not changing at all (especially on OC-3+) -> lower batch size / higher learning rate?

**Winner**: OC-3C

[comment]: <> (Execution Code: print_metrics_as_md "../reports/custom/prices-15-runs.csv")

## 2. 5 runs (bs=64)

- SimpleLSTM with seeds from 42 to 46
- On OC-3C and OC-3C+
- 60k train, 6k test
- 64 batch size

| -  |   datasets    predictors     |     OC-3C    SLSTM_b64_42   |     OC-3C    SLSTM_b64_43   |     OC-3C    SLSTM_b64_44   |     OC-3C    SLSTM_b64_45   |     OC-3C    SLSTM_b64_46   |     OC-3C+    SLSTM_b64_42   |     OC-3C+    SLSTM_b64_43   |     OC-3C+    SLSTM_b64_44   |     OC-3C+    SLSTM_b64_45   |     OC-3C+    SLSTM_b64_46   |
|---:|:-----------------------------|----------------------------:|----------------------------:|----------------------------:|----------------------------:|----------------------------:|-----------------------------:|-----------------------------:|-----------------------------:|-----------------------------:|-----------------------------:|
|  0 | prec                         |                     0.11111 |                     0.32211 |                     0.22711 |                     0.31561 |                     0.23353 |                      0.11111 |                      0.11111 |                      0.36631 |                            0 |                            0 |
|  1 | rec                          |                     0.33333 |                     0.34283 |                     0.33767 |                     0.3465  |                     0.33883 |                      0.33333 |                      0.33333 |                      0.37983 |                            0 |                            0 |
|  2 | f1                           |                     0.16667 |                     0.30668 |                     0.1786  |                     0.21627 |                     0.18374 |                      0.16667 |                      0.16667 |                      0.31404 |                            0 |                            0 |
|  3 | acc                          |                     0.33333 |                     0.34283 |                     0.33767 |                     0.3465  |                     0.33883 |                      0.33333 |                      0.33333 |                      0.37983 |                            0 |                            0 |
|  4 | mcc                          |                     0       |                     0.01561 |                     0.02991 |                     0.04257 |                     0.03177 |                      0       |                      0       |                      0.08696 |    

**Outcome**: Same problems. Even worse results for OC-3C

**Winner**: -

[comment]: <> (Execution Code: print_metrics_as_md "../reports/custom/prices-5-runs-bs64.csv")

## Current Execution

In [ ]:
datasets = [
    NyseStocksDataset('OC-3C', file_path='../data/nyse/prices.csv'),
    NyseStocksDataset('OC-3C+', file_path='../data/nyse/prices-split-adjusted.csv'),
]
[ds.load() for ds in datasets];

In [ ]:
def get_predictors(n_features, n_timestamps=7, n_classes=3):
    seeds = list(range(42, 47))
    return [
        SimpleLSTM(f'_b16_{s}', epochs=200, n_timestamps=n_timestamps, n_features=n_features, n_classes=n_classes, seed=s, batch_size=16)
        for s in seeds
    ]
evaluator = Evaluator('prices-5-runs-b16', datasets, get_predictors, '../reports', seed=42, store=True,
                      n_train_samples=60000, n_test_samples=6000)
metrics = evaluator()

In [ ]:
# evaluator.export_results()
# evaluator._metrics.to_csv(os.path.join(evaluator.output_dir, f'custom/{evaluator.name}.csv'))
# evaluator.plot_histories()
# evaluator.plotter.plot_metrics(evaluator._metrics, store=False);